In [ ]:
# RGB 2448x2448
# Urban land: 0,255,255: Light blue
# Agriculture land: 255,255,0: Yellow
# Rangeland: 255,0,255: purple
# Forest land: 0,255,0: light green
# Water: 0,0,255: blue
# Barren land: 255,255,255: white
# Unknown: 0,0,0: black

In [1]:
import os
from PIL import Image
import tensorflow as tf
from patchify import patchify, unpatchify
import numpy as np
import cv2

2025-03-29 13:23:41.152118: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 13:23:41.152759: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 13:23:41.158028: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 13:23:41.171120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743251021.193972   26291 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743251021.19

In [ ]:
image = Image.open("dataML/train/119_mask.png")
image.show()

image2 = Image.open("dataML/train/119_sat.jpg")
image2.show()

In [2]:
patch_size = 256
image_dataset = []
DataMLpath = "dataML/"

In [2]:
def patch_image(image):
    x, y = image.size

    x_2 = x//256
    y_2 = y//256

    cropped = image.crop((x-x_2, y-y_2, x, y))

    cropped = np.array(cropped)

    patched = patchify(cropped, (patch_size, patch_size, 3), step=patch_size)

    return patched

4


In [ ]:
# Crop images 2448x2448 -> 2304x2304
#--------- IMPORTANTE, NO EJECUTAR DE MOMENTO-----------

for root, dirs, files in os.walk(DataMLpath):
    
    if("train" in dirs):

        path = f"{root}train"
        images = os.listdir(path)

        for i, image_name in enumerate(images):
            
            image = cv2.imread(path+"/"+image_name, 1)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

            patched = patch_image(image)

            for i in range (patched.shape[0]):
                for j in range (patched.shape[1]):
                    patch = patched[i, j, 0]
                    patch_image = Image.fromarray(patch)

                    patch_image.save(os.path.join(f"{DataMLpath}train_patched", f"{image_name}_{i}_{j}.png"))

In [ ]:
#Original-->  imagename_sat.jpg_x_y
#After -->    imagename_mask.png_x_y

In [ ]:
    #U-NET ARCHITECTURE

def unet_model(input_shape=(256, 256, 3)):
    inputs = tf.keras.layers.Input(input_shape)

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    #Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)

    model = tf.keras.model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.summary()

    return model

In [ ]:
array = []

for root, dirs, files in os.walk(DataMLpath):

    if("train_patched" in dirs):
        path = f"{root}train_patched"
        images = os.listdir(path)

        for i, image_name in enumerate(images):
            string_path = f"{path}/{image_name}"
            
            image = cv2.imread(string_path)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            image = np.array(image)
            
            array.append(image)

In [ ]:
def rebuild_patched(predicted, original):
    return unpatchify(predicted, original.shape)


In [ ]:
def predict_input():
    for root, dirs, files in os.walk(DataMLpath):
        if("input" in dirs):
            path = (f"{root}input")
            images = os.listdir(path)

            for i, image_name in enumerate(images):
                image_path = f"{path}/{image_name}"
                image = cv2.imread(image_path)

                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image)

                patched = patch_image(image)

                #Predict every photo in patched
                #predicted = model.predict()
                predicted = []
                
                rebuilt = rebuild_patched(predicted, image)